In [ ]:
# CS194-26 (CS294-26): Project 1 starter Python code

In [51]:
#main
import numpy as np
import skimage as sk
from skimage import transform, filters
import skimage.io as skio


#normalized cross-correlation (NCC)
def normalize(z):
    return z - np.mean(z)

def NCC(x, y):
    nx = normalize(x)
    ny = normalize(y)
    normsum = np.sum(x * y)
    sqrtvar = np.sqrt(np.sum(x**2)) * np.sqrt(np.sum(y**2))
    
    return normsum/(sqrtvar) 

#ncc used for pyramid context
def align_ncc_pyramid(x, y, vert, hor):
    max_diff =  -1
    applied_i = 0;
    applied_j = 0;
    for i in range(vert - 5,vert + 5):
        for j in range (hor - 5, hor + 5):
            ri = np.roll(x, i, axis = 0)
            rj = np.roll(ri, j, axis = 1)
            ncc = NCC(rj, y)
            if (ncc > max_diff):
                applied_i = i
                applied_j = j
                max_diff = ncc
    return(applied_i, applied_j)
         
#the base method used in the pyramid - also used for smaller images
def align_ncc_pyramid_base(x, y):
    max_diff =  -1
    applied_i = 0;
    applied_j = 0;
    for i in range(-15,15):
        for j in range (-15, 15):
            ri = np.roll(x, i, axis = 0)
            rj = np.roll(ri, j, axis = 1)
            ncc = NCC(rj, y)
            if (ncc > max_diff):
                applied_i = i
                applied_j = j
                max_diff = ncc
    return(applied_i, applied_j)

#searches images from coarse to fine grain
def pyramid(x, y, n, stacks):
    if n == stacks:
        return align_ncc_pyramid_base(x, y)
    else:
        x = sk.transform.rescale(x, 0.75)
        y = sk.transform.rescale(y, 0.75)
        i, j = pyramid(x, y, n + 1, stacks)
        i = int(1.5 * i)
        j = int(1.5 * j)
        return align_ncc_pyramid(x, y, i, j)
    
#uses the roberts filter to autocrop images
def autocrop(b, g, r):
    #monastery
    b_i = autocrop_get_i(b)
    g_i = autocrop_get_i(g)
    r_i = autocrop_get_i(r)
    
    top_i = np.max([b_i[0],g_i[0],r_i[0]])
    bottom_i = np.max([b_i[1],g_i[1],r_i[1]])
    left_i = np.max([b_i[2],g_i[2],r_i[2]])
    right_i = np.max([b_i[3],g_i[3],r_i[3]])

    return  b[top_i:-bottom_i, left_i:-right_i], g[top_i:-bottom_i, left_i:-right_i], r[top_i:-bottom_i, left_i:-right_i]
    
    
#apply the ncc and pyramid methods to the proper channels, then  stack them
def align(b, g, r):
    
    stacks = 0
    height = np.floor(b.shape[0] / 3.0).astype(np.int)
    if height > 500:
        stacks = int(np.log(height) / np.log(1.5)) - 10
    print("stacks: " + str(stacks))

    ig, jg = pyramid(g, b, 0, stacks)
    print()
    ag = np.roll(np.roll(g, ig, axis = 0), jg, axis = 1)

    ir, jr = pyramid(r, b, 0, stacks)
    ar = np.roll(np.roll(r, ir, axis = 0), jr, axis = 1)

    # create a color image
    im_out = np.dstack([ar, ag, b])
    print("Displacement(x, y): R(" + str(ir) + ", " + str(jr) + "), G(" + str(ig) + ", " + str(jg) + "), B(0, 0)")
    return im_out

#save the new stacked image
def save(im_out, imname):
    # save the image
    fname = 'outputs/auto_crop/ac_' + imname[:-4] + ".jpg"
    skio.imsave(fname, im_out)

    # display the image
    skio.imshow(im_out)
    skio.show()
    
#get the amount need to crop each photo
def autocrop_get_i(image):
    
    image = filters.sobel(image)
    
    #set abritary boundary for cropping 
    max_border_vert = image.shape[0] // 15
    max_border_hor = image.shape[1] // 15 
    
    #get the difference between each  row or column
    top =  np.array([np.sum(image[i] - image[i-1]) for i in range(1,max_border_vert)])
    bottom = np.array([np.sum(image[-i] - image[-i-1]) for i in range(1,max_border_vert)])
    left = np.array([np.sum(image[:,i] - image[:,i-1]) for i in range(1,max_border_hor)])
    right = np.array([np.sum(image[:,-i] - image[:,-i-1]) for i in range(1,max_border_hor)])
    
    #want the cut off to be a big positive change (black to color)
    top_i = get_i(top)
    bottom_i = get_i(bottom)
    left_i = get_i(left)
    right_i = get_i(right)
        
            
    return top_i,bottom_i,left_i,right_i

def get_i(arr):
    ind = 0

    count = np.sum(arr < -20)
    
    #get the most recent biggest change in color
    #this is useful for when photos have both a black and white border
    for i in range(count):
        ind = np.argmin(arr)
        arr[ind] = 100
    return ind